In [14]:
from pymatgen.ext.matproj import MPRester
from pymatgen.symmetry.analyzer import *
from pymatgen.analysis.diffraction.xrd import XRDCalculator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from math import *
import numpy as np
with MPRester("7kc9QdoJwMtBxzFK") as m:
    print("Mon matériau est le CsPrS2")
    structure = m.get_structure_by_material_id("mp-9080")
    structure.to(filename="CsPrS2.cif")
    print()
    
    
    bs = m.get_bandstructure_by_material_id("mp-9080")
    bg = bs.get_band_gap()
    bdg = bs.get_direct_band_gap()
    print(bg)
    print(bg["energy"])
    print(bs.bands)
    cbm = bs.get_cbm()
    print(cbm)
    vbm = bs.get_vbm()
    print(vbm)
    print("La bande interdite représente un gap d'énergie de ",bg["energy"]," eV. Ce gap est indirect ce qui signifie que les deux valeurs d'énergie associées ne peuvent être associées au même vecteur d'onde.")
    print("(Si on avait uniquement considéré le gap direct il aurait eu une valeur de ",bdg," eV.)")

Mon matériau est le CsPrS2

{'direct': False, 'energy': 2.1610000000000005, 'transition': '\\Gamma-F'}
2.1610000000000005
{<Spin.up: 1>: array([[-33.1931, -33.1929, -33.1926, ..., -33.1762, -33.1762, -33.1762],
       [-18.9905, -18.9898, -18.9876, ..., -18.8943, -18.8945, -18.8946],
       [-15.4034, -15.4026, -15.4002, ..., -15.4816, -15.4813, -15.4808],
       ...,
       [ 12.137 ,  12.1066,  12.0342, ...,  12.5824,  12.5921,  12.6351],
       [ 13.079 ,  13.1208,  13.2151, ...,  13.3968,  13.4431,  13.3952],
       [ 13.8247,  13.8564,  13.9055, ...,  13.6323,  13.7507,  13.9772]])}
{'band_index': defaultdict(<class 'list'>, {<Spin.up: 1>: [16]}), 'kpoint_index': [91, 92], 'kpoint': <pymatgen.electronic_structure.bandstructure.Kpoint object at 0x7fadbda8ee80>, 'energy': 3.9761, 'projections': {}}
{'band_index': defaultdict(<class 'list'>, {<Spin.up: 1>: [14, 15]}), 'kpoint_index': [0, 59, 60], 'kpoint': <pymatgen.electronic_structure.bandstructure.Kpoint object at 0x7fadbdc615c0>,